In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, broadcast, desc, collect_list

from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, TimestampType, LongType, IntegerType, StringType
import os

In [6]:
spark = (
    SparkSession
    .builder
    .appName('Streaming III')
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0")
    .getOrCreate()
)

In [9]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

users_with_tag_output_path = os.path.join(project_path, 'output/users_with_tag')

In [11]:
users_with_tags = (    
    spark
    .read
    .option('path', users_with_tag_output_path)
    .load()
).cache()

AnalysisException: 'Unable to infer schema for Parquet. It must be specified manually.;'

In [3]:
def get_relevant_users(users_with_tags, tags_df, n_users):
    return (
        users_with_tags
        .join(broadcast(tags_df))
        .withColumnRenamed('tags', 'b')
        .selectExpr(
            'user_id',
            "FILTER(tag_info, x -> array_contains(b, x.tag)) AS new_tag"
        )
        .withColumn('tag_frequencies', col('new_tag.frequency'))
        .selectExpr(
            'user_id',
            "AGGREGATE(tag_frequencies, CAST(0 AS long), (value, buffer) -> value + buffer) AS question_relevancy"
        )
        .orderBy(desc('question_relevancy'))
        .limit(n_users)
        .agg(collect_list('user_id').alias('users'))
    )

In [ ]:
get_relevant_users(users_with_tags, new_question_tags, 10).show()

In [ ]:
stream_schema = StructType(
    [
        StructField('question_id', LongType()),
        StructField('tags', ArrayType(StringType()))
        
    ]
)

In [4]:
dx = ( 
    spark 
    .readStream 
    .format("kafka") 
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "test")
    .load()
)

x = (
    dx
    .selectExpr("CAST(value AS STRING) AS v")
    .select(from_json(col('v'), stream_schema))
)



NameError: name 'spark' is not defined

In [ ]:
(
    get_relevant_users(users_with_tags, x, 10)
    .writeStream
    .format('memory')
    .outputMode('append')
    .queryName('my_stream')
    .start()
)